# Importing libraries

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
import os, sys, glob, shutil
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'/Users/shubham/work/hurricane_damage_prediction'

In [13]:
from geocodio import GeocodioClient
from geocodio.exceptions import GeocodioDataError

# Loading and processing data

In [5]:
address = pd.read_csv("data/addresses_and_losses.csv")

/Users/shubham/anaconda3/envs/hurricane/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (43,46,47,50,55,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
address.head()

,applicant_type,Individual_Address,Individual_City,Individual_State,Individual_Zip,Business_Address,Business_City,Business_State,Business_Zip,Disaster_Id,...,FF_UP52_V,LEI_UP53_V,VEH_UP54_V,VES_UP55_V,ACF_UP56_V,RR_UP58_V,RM_UP59_V,WC_UP60_V,WC_UP61_V,NP_UP62_V
0,Household,314 CUT OFF ROAD,PORT ARANSAS,TX,78373.0,NaN,NaN,NaN,NaN,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Business,NaN,NaN,NaN,NaN,314 CUT OFF RD,PORT ARANSAS,TX,78373.0,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Household,5901 HORIZON DRIVE,CORPUS CHRISTI,TX,78408.0,NaN,NaN,NaN,NaN,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Household,622 JAMES ST,REFUGIO,TX,78377.0,NaN,NaN,NaN,NaN,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Household,RV PARK AVE # 44,CORPUS CHRISTI,TX,78401.0,NaN,NaN,NaN,NaN,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
damage_data = address[["applicant_type", "Individual_Address", "Individual_City", "Individual_State", "Individual_Zip", "Orig_RE_Verified_Loss"]]

Removing rows which consists of NA values. Considering NA values to be "not recorded" for now. Further analysis might be needed to check if NA Loss means 0 loss.

In [8]:
damage_data.dropna(subset=['Individual_Address', 'Orig_RE_Verified_Loss'], inplace=True)

/Users/shubham/anaconda3/envs/hurricane/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
damage_data.head()

,applicant_type,Individual_Address,Individual_City,Individual_State,Individual_Zip,Orig_RE_Verified_Loss
0,Household,314 CUT OFF ROAD,PORT ARANSAS,TX,78373.0,58999.0
2,Household,5901 HORIZON DRIVE,CORPUS CHRISTI,TX,78408.0,2563.0
7,Household,1202 MESQUITE ST SPACE 546,FULTON,TX,78358.0,26823.0
9,Household,314 DRISCOLL ST,WOODSBORO,TX,78393.0,99357.0
13,Household,3301 MEADOWLANE ST,VICTORIA,TX,77901.0,55813.0


In [14]:
client = GeocodioClient("0c410ad5ac00a2e7a055ade70475c465605a6ed")

In [17]:
geocoded_location = client.geocode("1202 MESQUITE ST SPACE 546, FULTON, TX")

In [20]:
geocoded_location.best_match

{'address_components': {'number': '1202',
  'street': 'Mesquite',
  'suffix': 'St',
  'formatted_street': 'Mesquite St',
  'city': 'Fulton',
  'county': 'Aransas County',
  'state': 'TX',
  'zip': '78358',
  'country': 'US'},
 'formatted_address': '1202 Mesquite St, Fulton, TX 78358',
 'location': {'lat': 28.066802, 'lng': -97.045522},
 'accuracy': 1,
 'accuracy_type': 'rooftop',
 'source': 'Statewide Partial'}

In [21]:
geocoded_location.accuracy

1

In [35]:
count = 0
def geocode(x):
    global count
    count+=1
    if count % 100 == 0:
        print(count)
    str_address = x.Individual_Address + ", " + x.Individual_City + ", " + x.Individual_State + ", " + str(int(x.Individual_Zip))
    geocoded_location = client.geocode(str_address)
    return((geocoded_location.coords, geocoded_location.best_match['formatted_address'], geocoded_location.accuracy,
            geocoded_location.best_match['accuracy_type']))

In [36]:
damage_data[['coordinates', 'formatted_address', 'accuracy', 'accuracy_type']] = damage_data.apply(lambda x: geocode(x), axis=1, result_type='expand')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


GeocodioAuthError: 

In [37]:
damage_data.head()

,applicant_type,Individual_Address,Individual_City,Individual_State,Individual_Zip,Orig_RE_Verified_Loss
0,Household,314 CUT OFF ROAD,PORT ARANSAS,TX,78373.0,58999.0
2,Household,5901 HORIZON DRIVE,CORPUS CHRISTI,TX,78408.0,2563.0
7,Household,1202 MESQUITE ST SPACE 546,FULTON,TX,78358.0,26823.0
9,Household,314 DRISCOLL ST,WOODSBORO,TX,78393.0,99357.0
13,Household,3301 MEADOWLANE ST,VICTORIA,TX,77901.0,55813.0


In [80]:
len(damage_data)

53565

In [30]:
small = damage_data.head(10)

In [32]:
small[['coordinates', 'formatted_address', 'accuracy']] = small.apply(lambda x: geocode(x), axis=1, result_type='expand')

314 CUT OFF ROAD, PORT ARANSAS, TX, 78373
5901 HORIZON DRIVE, CORPUS CHRISTI, TX, 78408
1202 MESQUITE ST SPACE 546, FULTON, TX, 78358
314 DRISCOLL ST, WOODSBORO, TX, 78393
3301 MEADOWLANE ST, VICTORIA, TX, 77901
189 PRESA ST, BEEVILLE, TX, 78102
7403 SEMINOLE ST, BAYTOWN, TX, 77521
291 COUNTY ROAD   # 4801, DAYTON, TX, 77535
3060 LENORE AVE, INGLESIDE, TX, 78362
708 RHINEGOLD ST, VICTORIA, TX, 77901


/Users/shubham/anaconda3/envs/hurricane/lib/python3.6/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [33]:
small

,applicant_type,Individual_Address,Individual_City,Individual_State,Individual_Zip,Orig_RE_Verified_Loss,coordinates,formatted_address,accuracy
0,Household,314 CUT OFF ROAD,PORT ARANSAS,TX,78373.0,58999.0,"(27.835148, -97.070685)","314 Cut-off Rd, Port Aransas, TX 78373",1
2,Household,5901 HORIZON DRIVE,CORPUS CHRISTI,TX,78408.0,2563.0,"(27.801748, -97.478782)","5901 Horizon Dr, Corpus Christi, TX 78408",1
7,Household,1202 MESQUITE ST SPACE 546,FULTON,TX,78358.0,26823.0,"(28.066802, -97.045522)","1202 Mesquite St, Fulton, TX 78358",1
9,Household,314 DRISCOLL ST,WOODSBORO,TX,78393.0,99357.0,"(28.235368, -97.331629)","314 Driscoll St, Woodsboro, TX 78393",1
13,Household,3301 MEADOWLANE ST,VICTORIA,TX,77901.0,55813.0,"(28.80527, -96.969475)","3301 Meadowlane St, Victoria, TX 77901",1
14,Household,189 PRESA ST,BEEVILLE,TX,78102.0,11663.0,"(28.387516, -97.776074)","189 Presa St, Beeville, TX 78102",1
15,Household,7403 SEMINOLE ST,BAYTOWN,TX,77521.0,77560.0,"(29.806424, -94.997548)","7403 Seminole St, Baytown, TX 77521",1
16,Household,291 COUNTY ROAD # 4801,DAYTON,TX,77535.0,151920.0,"(29.96275, -94.959057)","291 County Rd 4801, Dayton, TX 77535",1
17,Household,3060 LENORE AVE,INGLESIDE,TX,78362.0,52359.0,"(27.876735, -97.190915)","3060 Lenore Ave, Ingleside, TX 78362",1
18,Household,708 RHINEGOLD ST,VICTORIA,TX,77901.0,28851.0,"(28.792835, -97.016636)","708 Rhinegold St, Victoria, TX 77901",1


# Loading geo-coded data

In [42]:
data = pd.read_csv("data/geocoded_first_geocodio.csv")

TypeError: '<' not supported between instances of 'str' and 'float'